In [8]:
import os
import joblib
import io  
import base64  
import json
# Import necessary libraries  
import tensorflow as tf
from tensorflow.keras import backend as K  
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [5]:
model = tf.keras.models.load_model("model.keras", custom_objects={'dice_coef': dice_coef, 'iou': iou})

In [21]:
def dice_coef(y_true, y_pred, smooth=1e-6):  
    y_true_f = K.flatten(y_true)  
    y_pred_f = K.flatten(y_pred)  
    intersection = K.sum(y_true_f * y_pred_f)  
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)  
  
def iou(y_true, y_pred, smooth=1e-6):  
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])  
    union = K.sum(y_true,[1,2,3]) + K.sum(y_pred,[1,2,3]) - intersection  
    return K.mean((intersection + smooth) / (union + smooth), axis=0)  

def read_image(img_path):  
    x = cv2.imread(img_path, cv2.IMREAD_COLOR)  
    x = cv2.resize(x, (512, 256))  
    x = x/255.0  
    x = x.astype(np.float32)  
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    return x  


def model_inference_with_display(model, base64_image):  
    # Load and preprocess the image  
    image = read_image(base64_image)  
    image_to_predict = np.expand_dims(image, axis=0)
    
    prediction = model.predict(image_to_predict)  
    predicted_mask = tf.argmax(prediction, axis=-1)[0].numpy()  # Ensure it's a numpy array  
    predicted_mask_normalized = predicted_mask.astype('float32') / predicted_mask.max()  
    colored_mask = plt.get_cmap('viridis')(predicted_mask_normalized)[:, :, :3]  # Exclude alpha channel  
    colored_mask = (colored_mask * 255).astype(np.uint8)  # Convert back to 8-bit format  
  
   # Convert numpy array back to PIL image  
    mask_image = Image.fromarray(colored_mask)  
      
    # Convert the PIL Image to a bytes object in PNG format  
    buffered = io.BytesIO()  
    mask_image.save(buffered, format="PNG")  
    image_png_bytes = buffered.getvalue()  
      
    # Encode bytes to base64 string  
    base64_image = base64.b64encode(image_png_bytes).decode("utf-8")
    
     # Initialize a dictionary to hold polygons for each label  
    labels_polygons = {  
        'void': [],   
        'flat': [],            
        'construction': [],    
        'object': [],         
        'nature': [],         
        'sky': [],         
        'human': [],            
        'vehicle': []          
    }
    
    for label, value in labels_polygons.items():  
        label_index = list(labels_polygons.keys()).index(label)  # Get the index of the label  
        mask = (predicted_mask == label_index).astype(np.uint8) * 255  # Create a mask for current label  
          
        # Find contours  
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  
          
        for contour in contours:  
            approx = contour.flatten().tolist()  # Without approximation  
            labels_polygons[label].append(approx)  
      
    # Convert the dictionary to a JSON object  
    polygons_json = json.dumps(labels_polygons) 
    return polygons_json
    # return base64_image

In [22]:
def save_polygons_to_img(polygons_json, filename):  
    # Convert JSON string back to dictionary  
    polygons = json.loads(polygons_json)  
      
    # Initialize a black canvas/image  
    # Adjust `shape` based on the expected output size  
    shape = (256, 512, 3)  # Example shape, should match your actual image size  
    image = np.zeros(shape, dtype=np.uint8)  
      
    # Assign each label a unique color using viridis colormap  
    n_labels = len(polygons)  
    viridis = plt.get_cmap('viridis', n_labels)  # Get n_labels distinct colors from viridis  
      
    for i, (label, contours) in enumerate(polygons.items()):  
        color = viridis(i)[:3]  # Exclude alpha channel  
        color = tuple(int(c * 255) for c in color[::-1])  # Convert to BGR and scale to 0-255  
          
        for contour in contours:  
            contour = np.array(contour).reshape((-1, 1, 2)).astype(np.int32)  # Reshape and convert to needed format  
            cv2.fillPoly(image, [contour], color)  
      
    # Save the image  
    cv2.imwrite(filename, image)  

In [23]:
polygons = model_inference_with_display(model, "test.png")
save_polygons_to_img(polygons,"mask.png")

1/1 [==============================] - 3s 3s/step
iVBORw0KGgoAAAANSUhEUgAAAgAAAAEACAIAAABK8lkwAAAYUElEQVR4nO3dTWgeR57H8dKDhT0X2ZCAsFnjJFozi05acvH4ZBgjk0CYi1lmvBAMA2azpznsYS8OIT7MHHKY0yYYAtnAaIYll2EgIYKAT44uBp0eGII8MV5sDDHEusRGrNhDOTXt7n6qq6vrvb4fQpAfSc/Tel7+v3rr6qWLS5cFAKA+s9gHAAAY5/718+r/rVvUfyb3s3Tmv3/r6RCrsv7Kg9iHAIww//ZU85/yDdy8cdFb+u5XrwohXvv533weXbEOLjwc/BnD2m3t9I3b6msCwD3CAIlrVX9p/ZUH5gFwfO/wpWv3PB1eJbph4Lv0KyoDCAC/CANE11vuTfS+e1UACCHIAB/kM+xPswcwO3v1jtcHq9z821PyP/V17CMCTGnerk/WZvofgLWQw2tHgj1S5dRHRX5BzwBhTKzR829P5fVefXzzjPyC3okJAgCADdUJSCEhVN1v3kIGDCIAgGI5GaKJVeJN2vLduo9R4gTAsb2jT9eeRXnoFKTQYgLM6cctVcw4fGMfXHi4Ih4KIfavnGt9y7Do59v89z0J3BQhAI7tHQ3/oElJpNeMsgWYoW215Fy9sZvrI1e2dsS1k/Lrie39NAeFQpb7LoaA4iADUIBuP975G3v51km7X5RpISt+giNFceu+EiEAnq49oxMgWA4En3w0/+V9HrN9aPO3+vKtkwcXHlqX/qZW6U+kE5BI9Rf0AKLzMX6KSixqQ6S5PH9U58BJ9ReNmYAEOwEpIABSQRLAXLPEt2IgzeovNTOgWZHVQI1cWurwTKhW3U+h+Z+UmRCCk4GTwpnD0Ot9V+TyVnl888zBhYcHFx6ubO1ofibkIQV2cOFhczOGuFgFlKjmmcP0CSAMJo3Sz4DV7WVhsLKz+HZ6sE3fBnE9gBLQVy